In [1]:
import os
import os.path
import glob
import numpy as np
import Image,leargist
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
np.random.seed(1)

In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
X = np.zeros((num_samples, 320))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        im = Image.open(img_file)
        im1 = im.resize((64,64),Image.ANTIALIAS); # for faster computation
        des = leargist.color_gist(im1)
        X[cnt] = des[0:320]
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 320)

In [5]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [6]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(X,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.830789, total=   0.1s
[CV] ........................... C=0.01, score=0.843710, total=   0.1s
[CV] ........................... C=0.01, score=0.860051, total=   0.1s
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.828244, total=   0.1s
[CV] ........................... C=0.01, score=0.847328, total=   0.2s
[CV] C=0.1 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s


[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.866582, total=   0.2s
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.888041, total=   0.2s
[CV] ............................ C=0.1, score=0.838422, total=   0.3s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.856234, total=   0.3s
[CV] ............................ C=0.1, score=0.847328, total=   0.2s
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    1.6s


[CV] C=100000.0 ......................................................
[CV] ........................... C=10.0, score=0.840967, total=   6.2s
[CV] C=100000.0 ......................................................
[CV] ........................... C=10.0, score=0.878018, total=   6.6s
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.862595, total=   6.6s
[CV] C=1000000.0 .....................................................
[CV] .......................... C=100.0, score=0.829517, total=   6.5s
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.851145, total=   6.8s
[CV] C=1000000.0 .....................................................
[CV] .......................... C=100.0, score=0.880559, total=   6.9s
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.890585, total=   7.1s
[CV] C

...

[CV] ..................... C=10000000.0, score=0.778626, total=   8.5s
[CV] .................... C=100000000.0, score=0.788804, total=   8.2s
[CV] .................. C=10000000000.0, score=0.784987, total=   7.0s
[CV] ..................... C=10000000.0, score=0.833333, total=   8.7s
[CV] ................... C=1000000000.0, score=0.788804, total=   8.0s
[CV] .................... C=100000000.0, score=0.833333, total=   8.2s
[CV] ................... C=1000000000.0, score=0.784987, total=   8.2s
[CV] .................... C=100000000.0, score=0.852605, total=   8.9s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   17.6s remaining:    1.5s


[CV] .................. C=10000000000.0, score=0.852605, total=   8.1s
[CV] .................. C=10000000000.0, score=0.788804, total=   7.8s
[CV] .................. C=10000000000.0, score=0.833333, total=   4.3s
[CV] .................. C=10000000000.0, score=0.778626, total=   4.4s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   19.3s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [7]:
top_model.cv_results_

{'mean_fit_time': array([ 0.09291978,  0.21055455,  0.86405182,  6.62377796,  6.91880755,
         7.74023943,  8.03909612,  7.76601644,  7.76565418,  7.97203875,
         7.97018485,  7.42036738,  6.33070679]),
 'mean_score_time': array([ 0.04727769,  0.03096399,  0.03348956,  0.04064121,  0.04815645,
         0.04217315,  0.04818869,  0.06120915,  0.06601162,  0.06981182,
         0.02773418,  0.0339282 ,  0.00790377]),
 'mean_test_score': array([ 0.84202493,  0.85932333,  0.8641567 ,  0.86466548,  0.85220046,
         0.80284915,  0.79191046,  0.8018316 ,  0.76036632,  0.78860341,
         0.80768252,  0.80768252,  0.80768252]),
 'mean_train_score': array([ 0.84221566,  0.86135823,  0.87293305,  0.88914993,  0.89334747,
         0.83884883,  0.8312776 ,  0.83839917,  0.77321871,  0.81474494,
         0.84463462,  0.84463462,  0.84463462]),
 'param_C': masked_array(data = [0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0 100000.0 1000000.0
  10000000.0 100000000.0 1000000000.0 

In [8]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.84202, std: 0.01159, params: {'C': 0.01},
 mean: 0.85932, std: 0.01713, params: {'C': 0.10000000000000001},
 mean: 0.86416, std: 0.01687, params: {'C': 1.0},
 mean: 0.86467, std: 0.01788, params: {'C': 10.0},
 mean: 0.85220, std: 0.02059, params: {'C': 100.0},
 mean: 0.80285, std: 0.05841, params: {'C': 1000.0},
 mean: 0.79191, std: 0.03464, params: {'C': 10000.0},
 mean: 0.80183, std: 0.04983, params: {'C': 100000.0},
 mean: 0.76037, std: 0.08929, params: {'C': 1000000.0},
 mean: 0.78860, std: 0.08963, params: {'C': 10000000.0},
 mean: 0.80768, std: 0.02964, params: {'C': 100000000.0},
 mean: 0.80768, std: 0.02964, params: {'C': 1000000000.0},
 mean: 0.80768, std: 0.02964, params: {'C': 10000000000.0}]

In [10]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 10.0} - score: 0.8647
